In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
df_samples = pd.read_csv("../config/Trans_AT_network_all/samples_curated.csv")
df_samples.shape

In [ ]:
df_gtdbtk = pd.read_csv("../data/processed/Trans_AT_network_all/tables/df_gtdb_meta.csv")
df_gtdbtk.shape

In [ ]:
df_samples_bgc_only = df_samples.copy()
df_samples_bgc_only = df_samples_bgc_only.merge(df_gtdbtk, left_on="genome_id", right_on="genome_id").drop(columns=["genome_id.1"])
df_samples_bgc_only = df_samples_bgc_only[~df_samples_bgc_only.bgc_id.str.startswith("BGC")]
df_samples_bgc_only

In [ ]:
def combine_genus_species(row, keep_format=False):
    if row['Species'] == 's__':
        organism = row['Genus'] + ' sp.'
    else:
        organism = row['Organism']
    if keep_format:
        return "s__" + organism.split("__")[-1]
    else:
        return organism.split("__")[-1]

df_samples_bgc_only['Organism'] = df_samples_bgc_only.apply(combine_genus_species, axis=1)
df_samples_bgc_only['Species'] = df_samples_bgc_only.apply(lambda row: combine_genus_species(row, keep_format=True), axis=1)
outfile = Path("../tables/00_Curated_trans-AT_PKS.csv")
outfile.parent.mkdir(parents=True, exist_ok=True)
df_samples_bgc_only.to_csv(outfile, index=False)